## Finding a Maximum Value

Initialize `maximum.s` with the code prepared in the book:

In [ ]:
# PURPOSE:  An assembly program to find the largest number from a list.
# INPUT:    None.
# OUTPUT:   Returns the highest number found.
# VARIABLES:
#           %edi holds an index for the list.
#           %eax holds a list item indexed from the list by %edi.
#           %ebx throughout the operation holds the largest number found.

.section .data
# Given list of numbers:
data_items:
    .long 3,67,34,222,45,75,54,34,44,33,22,11,66,0

.section .text
.global _start
# Start operation by loading the first element into the registers:
_start:
    movl $0, %edi
    movl data_items(,%edi,4), %eax
    movl %eax, %ebx

# Instructions to process the list and find the largest number:
start_loop:
    # Exit if the current number found so far is equal to 0:
    cmpl $0, %eax
    je loop_exit

    # Otherwise, load the next number into %eax:
    incl $edi
    movl data_items(,%edi,4), %eax

    # Jump back to the beginning if the number isn't bigger than the one held so far:
    cmpl %ebx, %eax
    jle start_loop

    # Otherwise, then it is bigger so load that number into %ebx and then jump back to the beginning. 
    movl %eax, $ebx
    jmp start_loop

# Exit instructions:
loop_exit:
    movl $1, %eax   # Perform a system call to exit program.
    int $0x80       # Interrupt program and be handled.

Now, assemble and link it with these commands:
- `as maximum.s -o maximum.o`
- `ld maximum.o -o maximum`

Now run it, and check its status:
- `./maximum`
- `echo $?`

These lines are the `data` section:
```assembly
data_items:	# These are the data items
	.long 3,67,34,222,45,75,54,34,44,33,22,11,66,0 
```
where:
- `.data_items` is a label that refers to the written location that follows it.
- a directive that starts with `.long` causes the assembler to reserve memory for the list of numbers defined with it.

Because `data_items` is a label, any time in our program where we need to refer to this address we can use the `data_items` symbol, and the assembler will substitute it with the address where the numbers start during assembly.

There are several different types of memory locations other than `.long` that can be reserved. The main ones include:
- `.byte`, which refers to bytes that take up one storage location for each number. These are limited to numbers between 0 and 255.
- `.int`, which refers to *integers* that take up two storage locations for each number. These are limited to numbers between 0 and 65535.
- `.long`, which refers to *long integers* take up four storage locations. This is the same amount of space the registers use, which is why they are used in this program. They can hold numbers between 0 and 4294967295.
- `.ascii`, a directive to enter in *characters* into memory. Characters each take up one storage location (they are converted into bytes internally). So, if you gave the directive `.ascii "Hello there\0"` then the assembler would reserve 12 storage locations/bytes for each character. 

Within that *string* of characters, the first byte would contain the numeric code for `H`, the second byte for `e`, and so on and so forth.

The last character is represented by `\0`, which is the terminating character (it will never display, it just tells other parts of the program that it's the end of the string of characters).

Letters and numbers that start with a backslash represent characters that are not typeable on the keyboard or easily viewable on the screen. For example:
- `\n` refers to the "newline" character which causes the computer to start output on the next line.
- `\t` refers to the "tab" character.

All of the letters in an `.ascii` directive should be in quotes.

In our example, the assember reserves 14 long integer locations one right after another. Since each long takes up 4 bytes, that means that the whole list takes up 56 bytes.

`.data_items` is used by the assembler to refer to the address of the first of these values.

> *Take note that the last data item in the list is a zero. I decided to use a zero to tell my program that it has hit the end of the list. No matter how I do it, I must have some method of determining the end of the list. The computer knows nothing; it can only do what it is told. It's not going to stop processing unless I give it some sort of signal. Otherwise, it would continue processing past the end of the list into the data that follows it, and even to locations where we haven't put any data. Such are how things are that are built from the ground up.*

Notice that we don't have a `.global` declaration for `data_items`. This is because we only refer to these locations within the program. No other file or program needs to know where they are located. This is in contrast to the `_start` symbol, which Linux needs to know where it is so that it knows where to begin the program's execution.

In the comments you will notice that we've marked some *variables* that we plan to use. A variable is a dedicated storage location used for a specific purpose, usually given a distinct name by the programmer. In this program, we have several variables:
- one to hold the current maximum number found.
- one to hold the position of the list we are currently examining, called the *index*.
- one to hold the current number being examined with said index.

In this case, we have few enough variables so we can hold them all in registers. In larger programs, you have to put them in memory, and then move them to registers when you are ready to use them.

In this program, we are using `%ebx` as the location of the largest item we've found. `%edi` is used as the *index* to the current data item we're looking at. 

Now, let's talk about what an index is. The data item number is the *index* of `data_items`. You'll notice that the first instruction we give to the computer is `movl $0, %edi`.

Now, the next instruction is tricky, but crucial to what we're doing. It says `movl data_items(,%edi,4), %eax`.

To understand this line, you need to keep several things in mind:
- `data_items` is/refers to the location number of the start of our number list.
- Each number is stored across 4 storage locations because we declared the list they're in using `.long`.
- `%edi` is holding 0 at this point.

So, basically what this line says is:
1. Start at the beginning of `data_items`.
2. Take the first item number (because `%edi` is 0).
3. Extract the number, which takes up four storage locations.

Then it stores that number in `%eax`. This is how you write *indexed addressing mode* instructions in assembly language. The instruction in a general form is `movl BEGINNINGADDRESS(,%INDEXREGISTER,WORDSIZE)`, which in our case:
- `data_items` was our beginning address.
- `%edi` was our index register.
- 4 was our word size.

> This topic is discussed further in the Section called *Addressing Modes*.

Very strange things would happen if we used a number other than 4 as the size of our storage locations. It's actually what's called a *multiplier*. Basically, the way it works is that you start at the location specified by `data_items`, then you add `%edi` * 4 storage locations, and retrieve the number there. Usually, you use the size of the numbers as your multiplier, but in some circumstances you'll want to do other things.

(TODO. If you want a review. Continue refactoring notes.)

Let's look at the next line:
	- `movl %eax, %ebx`
	- We have the first item stored in `%eax`. As the first item, it's the biggest so far.
	- We store it in `%ebx`, since that's where we are keeping the largest number we found.
	- Also, even though `movl` stands for *move*, it actualyl copies the values, so `%eax` and `5ebx` both contain the starting value.
		- The `l` in `movl` stands for `move long` since we are moving a value that takes up four storage locations.
- We have marked the starting location of the loop in the symbol `start_loop`. We have a single section of code (a loop) that we execute over and over again for every element in `data_items`.
- Then we have these instructions:
	- ```assembly
	  cmpl $9, %eax
	  je end_loop		
	  ```
	- The `cmpl` instruction compares the two values. This compare instruction affects a register not mentioned here, the `%eflags` register, also known as the status register.
	- The next line is a flow control instruction which says to *jump* to the `end_loop` location if the values that were just compared to are equal (that's what the `e` of `je` means).
		- It uses the status register to hold the value of the last comparison.
- We used `je`, but there are many jump statements that you can use:
	- `je` - Jump if the values were equal
	- `jg` - Jump if the second value was greater than the first value
	- `jge` - Jump if the second value was greater than or equal to the first value
	- `jl` - Jump if the second value was less than the first value
	- `jle` Jump if the second value was less than or equal to the first value
	- `jmp` - Jump no matter what. This does not need to be preceeded by a comparison.
	- The complete list is documented in Appendix B.
- The names of these symbols can be anything you want them to be, as long as they only contain letters and the underscore character (_).
	- The only one that is forced is _start, and possibly others that you declare with `.global`.
- If the last loaded element was not zero, we go on to the next instructions:
	- ```assembly
	  incl %edi
	  movl data_items(,%edi,4), %eax
	  ```
	- `incl` increments the value of `%edi` by one.
	- Then the `movl` is just like the one we did beforehand.
- Now `%eax` has the next value to be tested. So, let's test!
	- ```assembly
	  cmpl %ebx, %eax
	  jle start_loop
	  ```
- Otherwise, we need to record that value as the largest one:
	- ```assembly
	  movl %eax, %ebx
	  jmp start_loop
	  ```
- The exit call requires that we put our exit status in `%ebx`.